In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('tamil_movies.csv')

In [3]:
# Compute TF-IDF vectors for movie overviews
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Review Data'].values.astype('U'))


In [4]:
# Compute cosine similarity between all pairs of movies
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [5]:
# Define function to recommend movies based on a given movie title
def hybrid_recommendations(title, num_recommendations=10):
    # Find the index of the movie in the DataFrame
    idx = df.index[df['Title'] == title].tolist()[0]

    # Get the cosine similarity scores for all movies relative to the given movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies by their similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the titles and genres of the top recommended movies
    movie_indices = [i[0] for i in sim_scores[1:]]
    recommended_movies = df.iloc[movie_indices][['Title', 'Genre']].values.tolist()[:num_recommendations]

    # Return the recommended movies
    return recommended_movies

In [6]:
print(hybrid_recommendations("Madras"))

[['Ko', 'Action, Thriller'], ['Saalaiyoram', nan], ['Vaathi', 'Drama, Action'], ['Kannitheevu', 'Thriller, Mystery'], ['Ji', 'Action, Drama'], ['Mandela', 'Comedy, Drama'], ['Kanni Maadam', nan], ['David', 'Action, Drama'], ['Maiem', 'Thriller'], ['Thirupathisamy Kudumbam', 'Drama, Romance']]


In [24]:
pip install customtkinter


Note: you may need to restart the kernel to use updated packages.


In [34]:
import tkinter as tk
from tkinter import ttk
from ttkthemes import ThemedTk
from hybrid_recommendations import *

class MovieRecommendation(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title('Tamil Movie Recommendation System')
        self.master.geometry('500x400')
        self.style = ttk.Style()

        # Create the movie title label and input field
        self.title_label = ttk.Label(self.master, text='Enter a movie title:', font=('Arial', 14))
        self.title_label.pack(side=tk.TOP, pady=10)
        self.title_input = ttk.Entry(self.master, font=('Arial', 14))
        self.title_input.pack(side=tk.TOP)

        # Create the button to trigger the recommendation
        self.recommend_button = ttk.Button(self.master, text='Get Recommendations', command=self.recommend, style='AccentButton.TButton')
        self.recommend_button.pack(side=tk.TOP, pady=20)

        # Create the label to display the recommended movies
        self.recommendation_label = ttk.Label(self.master, text='', font=('Arial', 12), justify='left', anchor='w')
        self.recommendation_label.pack(side=tk.TOP, pady=20)

        # Create the footer label
        self.footer_label = ttk.Label(self.master, text='Movie Recommendation System ', font=('Arial', 10), anchor='w')
        self.footer_label.pack(side=tk.BOTTOM, fill='x', padx=10, pady=10)

        # Define the styles for the widgets
        self.style.configure('AccentButton.TButton', font=('Arial', 14), foreground='#000000', background='#0052CC')
        self.style.configure('Accent.TLabel', foreground='#0052CC')

    def recommend(self):
        # Get the movie title from the input field
        title = self.title_input.get()

        # Call the recommendation function from the recommendation program
        recommendations = hybrid_recommendations(title)

        # Display the recommended movies in the label
        text = 'Recommended Movies:\n\n'
        for movie in recommendations:
            text += f'{movie[0]} - {movie[1]}\n'
        self.recommendation_label.config(text=text, style='Accent.TLabel')

if __name__ == '__main__':
    root = ThemedTk(theme='sunvally')
    app = MovieRecommendation(root)
    app.pack(expand=True, fill='both')
    root.mainloop()